In [1]:
import $exec.spark_functionality

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark UI

import $exec.$                  

In [2]:
import org.apache.spark.sql.functions._ 

val defrag_textreuses = spark.read.parquet("s3a://textreuse-new-processed-data/defrag_textreuses.parquet")
defrag_textreuses.createOrReplaceTempView("defrag_textreuses")

//val defrag_pieces = register("defrag_pieces",spark.read.parquet("s3a://textreuse-new-processed-data/defrag_pieces.parquet")
//    .select("piece_id"))

val edges = spark.sql("""
SELECT piece1_id AS piece_id, piece2_id AS other_piece_id
FROM defrag_textreuses
WHERE piece1_id<20
UNION ALL
SELECT piece2_id AS piece_id, piece1_id AS other_piece_id
FROM defrag_textreuses
WHERE piece1_id<20
""").persist()

edges.createOrReplaceTempView("edges")

val cluster_counts_initial = spark.sql("""
SELECT piece_id, map_from_entries(collect_list((other_piece_id,1))) AS cluster_counts
FROM edges
GROUP BY piece_id
""").persist()

val clusters_initial = cluster_counts_initial
    .select("piece_id")
    .withColumn("cluster_id",col("piece_id"))

var active_initial = cluster_counts_initial
    .select("piece_id")

: 

In [10]:
var cluster_counts = cluster_counts_initial
var clusters = clusters_initial
var active = active_initial
var active_count = active.count()
var iter = 0

count at cmd9.sc:10 
 (kill)

0 / 200

count at cmd9.sc:10 
 (kill)

0 / 1

cluster_counts: Dataset[Row] = [piece_id: bigint, cluster_counts: map<bigint,int>]
clusters: DataFrame = [piece_id: bigint, cluster_id: bigint]
active: DataFrame = [piece_id: bigint]
active_count: Long = 23L
iter: Int = 0

In [11]:
while (active_count != 0 && iter<10) {
  iter += 1
  println(f"Starting iteration ${iter} with ${active_count} active nodes.")
  val new_cluster_updates = cluster_counts
    .join(active,"piece_id")
    .withColumn("new_cluster_id",
      expr("""
          aggregate(
            map_keys(cluster_counts), 
            (bigint(-1) as cluster_id, bigint(-1) as count, bigint(-1) as same_count),
            (acc, y) ->
              IF(
                acc.count <= cluster_counts[y],
                IF(
                  acc.count < cluster_counts[y],
                  (y as cluster_id, cluster_counts[y] as count, 1 as same_count),
                  IF(
                    rand()<1/(acc.same_count + 1),
                    (y as cluster_id, acc.count as count, acc.same_count + 1 as same_count),
                    (acc.cluster_id as cluster_id, acc.count as count, acc.same_count + 1 as same_count)
                  )
                ),
                acc
              )
          )
      """))
    .select("piece_id","new_cluster_id.cluster_id","new_cluster_id.same_count")
    .withColumnRenamed("cluster_id","new_cluster_id")
  val count_updates = clusters
    .withColumnRenamed("cluster_id","old_cluster_id")
    .join(new_cluster_updates,"piece_id")
    .select("piece_id","old_cluster_id","new_cluster_id")
    .filter((col("old_cluster_id")!==col("new_cluster_id")) && (rand()<=0.9))
    .persist()
  val projected_count_updates = count_updates
    .join(edges,"piece_id")
    .select("other_piece_id","old_cluster_id","new_cluster_id")
    .withColumnRenamed("other_piece_id","piece_id")
    .groupBy("piece_id")
    .agg(collect_list(map(col("old_cluster_id"),lit(-1),col("new_cluster_id"),lit(1))).as("count_updates"))
    .withColumn("count_updates",expr("""
      aggregate(
        count_updates,
        cast(map() AS MAP<BIGINT, BIGINT>),
        (acc, y) -> map_zip_with(acc,y, (k, v1, v2) -> coalesce(v1,0) + coalesce(v2,0))
      )
    """))
    .persist()    
  active = projected_count_updates
    .select("piece_id")
    .union(
      new_cluster_updates
        .filter(col("same_count")>lit(1))
        .select("piece_id")
    )
    .distinct()
    .checkpoint()
  active_count = active.count()
  if (!count_updates.isEmpty) {
    clusters = clusters.as("cl"+iter)
      .join(count_updates,clusters("piece_id")===count_updates("piece_id"),"left")
      .withColumn("cluster_id",coalesce(col("new_cluster_id"),col("cluster_id")))
      .select("cl"+iter+".piece_id","cluster_id")
      .checkpoint()
    cluster_counts = cluster_counts.as("cc")
      .join(projected_count_updates.as("cu"),col("cc.piece_id")===col("cu.piece_id"),"left")
      .withColumn("new_cluster_counts",
        expr("""
          IF(isnull(count_updates),
            cluster_counts,
            map_filter(
              map_zip_with(cluster_counts,count_updates, (k,v1,v2) -> coalesce(v1,0) + coalesce(v2,0)),
              (k,v) -> v!=0
            )
          )
        """))
      .select("cc.piece_id","new_cluster_counts")
      .withColumnRenamed("new_cluster_counts","cluster_counts")
      .checkpoint()
  }
}

Starting iteration 1 with 23 active nodes.


$anonfun$withThreadLocalCaptured$1 at FutureTask.java:264 
 (kill)

0 / 420

$anonfun$withThreadLocalCaptured$1 at FutureTask.java:264 
 (kill)

0 / 200

...

0 / 420

checkpoint at cmd10.sc:78 
 (kill)

0 / 200

checkpoint at cmd10.sc:78 
 (kill)

0 / 200

In [13]:
iter

res12: Int = 10